# Assignment 4 - Lorenzo Leuzzi

DATASET (LERCIO HEADLINES) - Dataset collected by Michele Cafagna

Pick up one of the available implementations of the Char-RNN (e.g. implement1,  implement2,  implement3, implement4, etc.) and train it on the dataset which contains about 6500 headlines from the Lercio satirical newspage, scraped by Michele Cafagna, past student of the ISPR course. The dataset is contained in a CSV file, one line per headlines. Be aware that the dataset can be a bit noisy (some errors due to encoding conversions) so you might need some preprocessing in order to prepare it for the task. Also, I am afraid the dataset is in Italian only as this is the language of the newspage.

Try experimenting with different configurations of the CHAR-RNN, varying the number of layers. Since the dataset is quite small, keep the number of hidden neurons contained otherwise the net will overfit. Use the trained model (the best or the worst, your choice) to generate new headlines.  

The softmax has a temperature parameter T that you can use to control the randomness of the output distribution (i.e. output logits are divided by T). Experiment with different values of T and comment the results.

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 10.7 MB/s eta 0:00:00


In [ ]:
import csv
import re
import unidecode
import string
import random
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse
import os
from tqdm import tqdm

DATASET_PATH = "/content/drive/MyDrive/ispr/lercio_headlines.csv"

In [ ]:
# Read the CSV file into a list of headlines
headlines = []
with open(DATASET_PATH, 'r') as f:
    reader = csv.reader(f)
    headlines = [row[0] for row in reader]

In [ ]:
headlines[:3]

['Nuovo slancio d’altruismo di Candreva: “Aiuterò Borja Valero ad attraversare la metà campo”',
 'Tre bacini ma non passa la bua. Mamma denunciata per malasanità',
 'Archeologia: nella prossima puntata Alberto Angela parlerà di altavista.com']

## Preprocessing

To **preprocess** our data we want to remove any duplicate headlines, remove any "unreadble character" and have all the headlines in lowercase.

In [ ]:
# Remove duplicate headlines
headlines = list(set(headlines))

# Remove non-ASCII characters from headlines
for i in range(len(headlines)):
    headlines[i] = unidecode.unidecode(headlines[i])

# Convert all headlines to lowercase
for i in range(len(headlines)):
    headlines[i] = headlines[i].lower()

In order to train the Char-RNN, we're going to be feeding the input data in batches and have a complete headline for each pattern. If the headlines have different lengths, that is not possible. By **padding** the headlines to a fixed length, we can efficiently create batches data that have a fixed size and we are sure that the headlines are fed correctly into the network.

In [ ]:
# Define the maximum length for headlines
max_length = len(max(headlines, key=len)) #168
#print(max_length)
# Define the padding character
padding_char = '|' # char not present in any headline

# Pad each headline to the maximum length
for i in range(len(headlines)):
    headline = headlines[i]
    if len(headline) < max_length:
        padding_length = max_length - len(headline)
        padded_headline = headline + padding_char * padding_length
        headlines[i] = padded_headline

In [ ]:
headlines[:3]

["maestro elementare di scilipoti condannato per crimini contro l'umanita|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||",
 'trump contro l\'iniezione letale: "troppo cara, la sostituiro con una sega arrugginita"||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||',
 'supermercato squalifica anziano per partenza anticipata|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||']

In [ ]:
N_HEADLINES = len(headlines)
print(N_HEADLINES)
len(headlines[0]) #168 length of all the headlines after padding

6485


168

Save the preprocessed headlines into a **text file** to be consistent with the choosen implementation.

In [ ]:
with open('headlines.txt', 'w') as f:
    for line in headlines:
        f.write(f"{line}\n")

## Model

I choose this [implementation](https://github.com/spro/char-rnn.pytorch) because it was the simplest to understand and it had all the necessary features needed. Here's the **CharRNN** model definition.

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def forward2(self, input, hidden):
        encoded = self.encoder(input.view(1, -1))
        output, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

## Training

### Utility functions

In [ ]:
# Readable time elapsed
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
def save(decoder, name):
    #save_filename = os.path.splitext(os.path.basename(args.filename))[0] + '.pt'
    save_filename = f'/content/drive/MyDrive/ispr/{name}.pt'
    torch.save(decoder, save_filename)
    print('Saved as %s' % save_filename)

In [ ]:
def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file

In [ ]:
# Turning a string into a tensor
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

In training a Char-RNN model, we typically do not feed the entire training dataset to the model in a single epoch. Instead, we break the dataset into smaller **chunks** of size `batch_size`, and randomly sample one of these chunks at each epoch. This ensures that the model is exposed to a diverse range of examples and reduces the time required for training.

In [ ]:
def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for batch_i in range(batch_size):
        random_hd_id = random.randint(0, N_HEADLINES-1)
        start_index = random_hd_id * (chunk_len + 1)

        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]

        try:
          inp[batch_i] = char_tensor(chunk[:-1])
          target[batch_i] = char_tensor(chunk[1:])
        except RuntimeError: # to understand which headline caused problems
          print(random_hd_id)
          print(f"{start_index}: {chunk}")

    inp = Variable(inp)
    target = Variable(target)

    return inp, target

### Generate

This function is used to generate text **predictions** from a trained Char-RNN decoder model.

In [ ]:
def generate(decoder, prime_str='a', predict_len=169, temperature=0.8):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)

    inp = prime_input[:,-1]

    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)

        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))

    return predicted

### Run training

We read the data from the previously created text file.

In [ ]:
# Reading and un-unicode-encoding data
all_characters = string.printable
n_characters = len(all_characters)
file = read_file("headlines.txt")
print(file[:168]) # show an headline

maestro elementare di scilipoti condannato per crimini contro l'umanita|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||


In [ ]:
def run_experiment(n_layers = 1, n_epochs = 1000, batch_size = 128):
  chunk_len = max_length
  print(chunk_len)

  # Initialize models
  hidden_size = 70
  model_type = "gru"

  learning_rate = 0.01
  decoder = CharRNN(
      n_characters,
      hidden_size,
      n_characters,
      model_type,
      n_layers=n_layers
  )

  decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss()

  def train_step(inp, target):
    hidden = decoder.init_hidden(batch_size)
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len-1):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data / chunk_len


  start = time.time()
  all_losses = []
  loss_avg = 0

  # Start training
  try:
      print("Training for %d epochs..." % n_epochs)
      for epoch in tqdm(range(1, n_epochs + 1)):
          loss = train_step(*random_training_set(chunk_len, batch_size))
          loss_avg += loss

          if epoch % 100 == 0:
              print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
              print(generate(decoder), '\n')

      print("Saving...")
      save(decoder, f"char-rnn{n_layers}")

  except KeyboardInterrupt:
      print("Saving before quit...")
      save(decoder, f"char-rnn{n_layers}")

Now we run different experiments varying the number of hidden layers. When the training is finished the models are saved in my drive.

In [ ]:
run_experiment(n_layers = 1, n_epochs = 2000)

168
Training for 2000 epochs...


  5%|▌         | 100/2000 [01:07<20:10,  1.57it/s]

 7s (100 5%) 0.9169]
a cambora i la in appalionaa|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



 10%|█         | 200/2000 [02:15<19:10,  1.57it/s]

 15s (200 10%) 0.8096]
azza e angiera da ritica del cresa brocersi appo depettona di casa il passi||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



...

 8s (1800 90%) 0.7196]
artive ai nuovo viscinisti in un rifetto del trimanche di sponte|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



 95%|█████████▌| 1900/2000 [21:15<01:04,  1.55it/s]

 15s (1900 95%) 0.6873]
andhia per riempresturista di emenelata a cacce un altre di salosita|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



100%|██████████| 2000/2000 [22:22<00:00,  1.49it/s]

 21s (2000 100%) 0.7279]
arrifo di casario da solo berluscone automula la maio del busta si una scosoni|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 

Saving...
Saved as /content/drive/MyDrive/ispr/char-rnn1.pt


In [ ]:
run_experiment(n_layers = 2, n_epochs = 2000)

168
Training for 2000 epochs...


  5%|▌         | 100/2000 [01:28<32:49,  1.04s/it]

 28s (100 5%) 0.9519]
ar oconbi con il silca nesza li malecer cala siosto, inni dla malina della per parcettera |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



 10%|█         | 200/2000 [02:57<28:03,  1.07it/s]

 57s (200 10%) 0.8768]
a, ablotto nell'ancordi labbbullo in 310co la mariana l'ardisi sulli minato||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



...

 95%|█████████▌| 1900/2000 [27:56<01:22,  1.21it/s]

 56s (1900 95%) 0.6402]
andro faranno una caglore ma al candeva ammaiso per riallo con piu faccia la compure|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



100%|██████████| 2000/2000 [29:25<00:00,  1.13it/s]

 25s (2000 100%) 0.6588]
altroma lo stragia: "nagone in cui non ci mandare"|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 

Saving...


Saved as /content/drive/MyDrive/ispr/char-rnn2.pt


In [ ]:
run_experiment(n_layers = 3, n_epochs = 2000)

168
Training for 2000 epochs...


  5%|▌         | 100/2000 [02:02<42:08,  1.33s/it]

 2s (100 5%) 0.9772]
acpiu selanre ptutarto gesFmissigii serio damalo nuantri pellotii ti dera per dearre soro sha nolresto|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



 10%|█         | 200/2000 [04:00<33:09,  1.11s/it]

 0s (200 10%) 0.8363]
amende daglo mere in miloekta di lencie su vendese al feole incina dell alita cenze di nefcieni greda e cuone||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



...

 6s (1800 90%) 0.6556]
antivoloyon di porta due incinlitati di renzi di stavolyaly aurica contenano dalla santa chiede di accusaggio||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



 95%|█████████▌| 1900/2000 [37:03<01:58,  1.18s/it]

 3s (1900 95%) 0.5920]
allarme, scopre conferma: vendono il confesso di puti mi se luzie||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| 



100%|██████████| 2000/2000 [38:58<00:00,  1.17s/it]

 58s (2000 100%) 0.6217]
aleni brasi la verda dalvale con sindario in artamia a casa in un cavologo di eorico in barza di mondiali di poteri non 10|||||||||||||||||||||||||||||||||||||||||||||||| 

Saving...
Saved as /content/drive/MyDrive/ispr/char-rnn3.pt


## Test the different models

We now test the different models generating headlines.

In [ ]:
prime_examples = ["scoperto", "oggi", "attacco", "due", "forza", "il", "uomo", "donna", "ritrovato", "governo", "salvini"]
predicted_headlines = []

for i in range(1, 4):
  model = f'/content/drive/MyDrive/ispr/char-rnn{i}.pt'
  decoder = torch.load(model)
  for prime in prime_examples:
    predicted = generate(decoder, prime)
    predicted_headlines.append(predicted.replace("|", ""))

Predicted headlines for the first model (1 hidden layer)

In [ ]:
predicted_headlines[:len(prime_examples)]

['scoperto al presetto di perto a maio solo a radio e a film costri fare incuita vota da una buone',
 "oggi d'a medice e non resterchete dall'ancora indive la concermenio",
 'attacco diavano in camano il cuore allo giovanne',
 "due a saranno tunivante di revista di torna in conestita: 'la prima i cascina a malice",
 "forza con che l'inventopegna dal 2019 non verdi a casa di capia di gioca",
 'il w70% il corrazzo a un lapollamentare: "sono luitente del noalo tutalia del divorziamo di rittorizzista occupe"',
 'uomo nuovo verna avantico rivela: "non scopista animarino all\'ambita di complavapo"',
 'donna risara la mestice al confretto: grande solo nel boccone. ai girette ascoltare cucle',
 'ritrovato infarto trova la chinestrilizione oratica la assetta finanza incidente dal 2027',
 'governo di vince a zodiarga il contro per scopatifono essere a magione da stato a renzi da miliandi ma il caporali',
 'salvini fa "l\'arrivo farato di mentata il paganzante dal morte della traduse chimica di t

Predicted headlines for the second model (2 hidden layers)

In [ ]:
predicted_headlines[len(prime_examples):len(prime_examples)*2]

["scoperto padre non prende il capiscole svando che lo spazzetta del palinon si non esarire l'esporta all'all'estotivissione",
 'oggiusta preceden attide alle arrestato in mangia e il fermio e un altro gli scienziati',
 'attacco, la collano a era volte gial alberi: "ci chianno i genistri di notti e il progestimo"',
 'due camina con gli preneare i ciolizzato: "ai nei finici e credeva imali"',
 'forzato gattina, ritrova rai dice di maggia',
 'il mio: "basti si ancorn disara per chiede l\'alt al mondo di cardi"',
 'uomo. "tracqua al regoli per i potremo la dura le fabbrica"',
 'donna bagno al pizzica il pace di chachia e fa riservie mondiale',
 'ritrovatore "rischus accusa delle salute mesi": 2 reddito di seritarno di terre del mondo cieli',
 'governo, alle costi come matte la mostra e fa esclusa esimisco e fa aderantini',
 "salvini anche il raccone gara la maggiorna a mente sono avvici in un contro l'uoma"]

Predicted headlines for the third model (3 hidden layers)

In [ ]:
predicted_headlines[len(prime_examples)*2:]

['scoperto ua sciolzare mi emeratori al parente cazzata intretta: storato su chavorico',
 'oggio la famiglia in parranza a gresi, suicidio veloce di sue mese del marito di applaus',
 'attacco di adotare 10 mesi: "odianna a 2 chi per stueomi della votano fencere"',
 'due confessa: "mi affancati andato al corona fa vedono sempre il goccorsare e fa un\'orare la ravermausa"',
 "forza esaganti tanti compra chiedere il profogorione dell'occhione e per il discotaro porta",
 'il paglia a un matrimonio omicidio di arrestata ma era guarisce banca',
 'uomo detta dal carcarestato 12 oreli di fine mondiale del mello di bari',
 'donna impossione in stato cerca anche sessuale e i visiti',
 'ritrovato il nuovo portico signotero le ricchi di mercanno su un sclater',
 'governo uccide la barca di processare le elezioni in 10mpra di mariatore',
 'salvini: "troppi meno dovra il tuo degli rotti?" per cosa vieta che giovane ottiene il primo bumminato 6']

From this examples it is not clear which model generates  better headlines (the third one might have a slight edge). Considering the training losses, the third model has the lowest value so we are going to choose it for the next section.

### Experimenting with temperature

We now experiment with different values of the softmax **temperature** parameter T that we can use to control the randomness of the output distribution.

In [ ]:
best_model = f'/content/drive/MyDrive/ispr/char-rnn3.pt'
best_decoder = torch.load(best_model)
temperatures = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

predictions_t = []
for t in temperatures:
  predicted = generate(best_decoder, temperature=t)
  predictions_t.append(predicted.replace("|", ""))

The results are shown in ascending temperature value

In [ ]:
predictions_t

['arriva la mano in italia di maio e una sciolda di arrestato da un mese per le compagni di santani',
 'allarme della lava a discorso dal carcere delle col commento di maio in compagno',
 'allarme del mondo al giorno per aver per i casalesi malieri alle spadani',
 "allarme scopre che l'omosessuale e dovemo merca le foto di parenti all'autobilitato",
 'arriva le mano al 2016 la famiglia di annuncia la testa comita il gastico del sedita di studio di prossimo di maria',
 "alfarme del con confossare che sbaglia l'italia e per bambine",
 'arriva le lavora il masilo di fedele di fermatella fisica',
 "also finisce internamiane su scopre che l'avemlas che non la fina dall'etremerisce a servi"]

From this results we can say that model generates more understandable headlines with lower values of the temperature parameter (0.3-0.5).

In [ ]:
best_predictions = []
for prime in prime_examples:
   predicted = generate(best_decoder, prime, temperature=0.35)
   best_predictions.append(predicted.replace("|", ""))

best_predictions

['scoperto il primo stato di colore per stagione e viene alla fila di roma',
 'oggiara di cambia le confessa: "il bambino della parte del figlio per le strade di maio"',
 'attacco di casa il mare di allevica per un commedio alle coppie in stato di euro per amare il mondo',
 'due mangia il cancella piu per dare i calcio di giorni alle come scopre che si fa la scheda',
 "forza scambia la madre all'ario a sosteneva e scopre di appendiamo alla paga",
 'il redazione si fa il prossimo alle completario e viene il primo contro male',
 'uomo paralico di corre di marie al commedista del mondo di salvini annuncia',
 'donna alla scarabilita di san solo una parte della barbona di maio alle mani di essere il mercato della fabio',
 'ritrovato il prossimo solo un cancella per le copie della parte e la pancia se scanzi a fare il campo di saranno al giorno di correr',
 'governo il capo di compassano il primo maggio a un confondo dopo le porta di vero',
 'salvini si compra le come solo in servizio e muor

## Conclusions

During the studying and the exploration of Recurrent Neural Networks, specifically the Char-RNN model, I found the experience to be both engaging and intriguing. Being a deep learning model, it has the potential to create new text data, and it was fascinating to observe how the model was able to partially learn and comprehend the patterns and structure of the original dataset to generate new text sequences. While the generated headlines were not entirely credible, they did reflect the satirical and comedic aspects of the original dataset (*Lercio*), which was a noticeable outcome.